In [82]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.layouts import column, row
#from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
import numpy as np



df = pd.read_csv('Titanic/Titanic-Dataset.csv')

#df.isna().sum()

# Handle missing values
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Cabin'].fillna('Unknown', inplace=True)
df['Embarked'].fillna('N/A', inplace=True)

df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [83]:
# - Create a new column AgeGroup to categorize passengers into age groups (e.g.,
# Child, Young Adult, Adult, Senior).
# - Create a SurvivalRate column to calculate the percentage of passengers who
# survived within each group.
df['AgeGroup'] = np.where(df['Age'] < 11, 'Child',
                    np.where(df['Age'] < 20, 'Young Adult', 
                             np.where(df['Age'] < 60, 'Adult', 'Senior')
                            )
                         )

# Group by Pclass, Sex, AgeGroup
grouped_df = df.groupby(['Pclass', 'Sex', 'AgeGroup'])['Survived'].mean().reset_index()
#display(grouped_df)

# Make percentage value
grouped_df['SurvivalRate'] = (grouped_df['Survived'] * 100).round(2)

# Merge SurvivalRate dataframe (grouped_df) with original dataframe (df)
df_survived = pd.merge(df, grouped_df[['Pclass', 'Sex', 'AgeGroup', 'SurvivalRate']], 
              on=['Pclass', 'Sex', 'AgeGroup'], how='left')

display(df_survived)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeGroup,SurvivalRate
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S,Adult,12.50
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Adult,97.40
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S,Adult,47.96
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Adult,97.40
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S,Adult,12.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,Unknown,S,Adult,7.06
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Young Adult,100.00
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,Unknown,S,Adult,47.96
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Adult,38.61


In [63]:
# import pandas as pd
# from bokeh.plotting import figure, show
# from bokeh.io import output_notebook
# from bokeh.models import ColumnDataSource, HoverTool
# from bokeh.transform import factor_cmap
# from bokeh.palettes import Spectral6

# # Output the plot to the notebook (if you're using a Jupyter Notebook)
# output_notebook()

Loading BokehJS ...

In [84]:
age_group_survival = df_survived.copy()


# Calculate survival rates for each AgeGroup
age_group_survival = df.groupby('AgeGroup')['Survived'].mean() * 100  # Convert to percentage
age_group_survival = age_group_survival.reset_index()

# Convert the categorical AgeGroup to a string list for Bokeh
age_group_survival['AgeGroup'] = age_group_survival['AgeGroup'].astype(str)

#display(age_group_survival)

source = ColumnDataSource(age_group_survival)

# Ensure the x_range is unique and ordered
unique_age_groups = age_group_survival['AgeGroup'].unique().tolist()

# Convert the data to a ColumnDataSource for Bokeh
source = ColumnDataSource(age_group_survival)


# Create the figure with string categories for x_range
p = figure(x_range=unique_age_groups, height=500, width=700, title="Survival Rates by Age Group",
           toolbar_location=None, tools="")

# Add bars to the plot
p.vbar(x='AgeGroup', top='Survived', width=0.9, source=source, color='skyblue')

# Customize the plot
p.y_range.start = 0
p.yaxis.axis_label = "Survival Rate (%)"
p.xaxis.axis_label = "Age Group"

# Show the plot
show(p)